
# Przygotowanie

Przygotowanie
Przed rozpoczęciem pracy z notatnikiem proszę zmienić jego nazwę dodając na początku numer albumu, imię i nazwisko. {nr_albumu}_{imię}_{nazwisko}_{nazwa}

Po wykonaniu wszystkich zadań proszę przesłać wypełniony notatnik przez platformę ELF za pomocą formularza "Prześlij projekt" w odpowiedniej sekcji.

# Support Vector Machine

Jest to jeden z najbardziej rozpowszechnionych i wszechstronnych modeli uczenia maszynowego. Z jego uzyciem dokonac mozna klasyfikacji liniowej (SVC), nieliniowej jak i regresji (SVR). Na poniższej grafice przedstawione zostało działanie klasyfikatora.

![svc](svc.png)

Analizujac grafike dostrzec mozna dwie oddzielne klasy oddzielone za pomoca prostej. Widoczna linia ciagła rozdziela klasy, a przerywane linie oznaczają margines, czyli możliwe najdalsze oddalenie elementu (np. nowego) jaki zakwalifikowany
zostanie do danej klasy. Maszyny SVM czułe sa na skale danych, przed ich uzyciem zawsze powinna zostać przeprowadzona normalizacja danych (np. min-max, lub standaryzacja).

![svc_example](svc2.jpg)

Równowage pomiedzy marginesami możemy regulować za pomoca hipermarapetru
C. Mniejsze jego wartości poszerzają granice, jednocześnie wprowadzając
więcej jej naruszeń. Im margines jest szerszy, tym własciwosci generalizujace
jakie posiada klasyfikator będę większe. Mniejsza staje się podatność na przeuczenie
(ang. overfitting), ale zmniejsza się skuteczność klasyfikatora. Szukany jest
taki klasyfikator, który podzieli przestrzeń na dwa rozłaczne zbiory odpowiadajace
dwóm klasom, w możliwie optymalny sposób. Podejście opiera się na
znalezieniu granicy decyzyjnej.

Wektory nośne (Support vectors) są to obserwacje (data points), które wystepują najbliżej hiperpłaszczyzny. Punkty te, pomagają lepiej wyznaczyć linię separacji pomiędzy klasami poprzez obliczenie marginesów. Są to najbardziej znaczace obserwacje ze zbioru z punktu widzenia konstrukcji klasyfikatora.

Warto zaznaczyć, że za pomocą klasyfikatora SVC można klasyfikaować dane, które nie są linowo separowalne. Można to osiągnąć przez tzw "sztuczkę kernelową", dzięki czemu możliwe jest zmapowanie obserwacji do wielowymiarowej przestrzeni. Klasyfikator z biblioteki Sklearn posiada parametr *kernel*, który pozwala na zmianę jądra. Dodatkowo, parametr *gamma* pozwala na modyfikację działania samego kernela.

Warto zaznaczyć, że SVC dobrze nadaje się do niewielkich zbiorów danych, gdyż w przypadku dużej ilości staję się on mało wydajny.

Funkcja jaka jest minimalizowana podczas działania klasyfikatora wygląda następująco:

\begin{equation}
min C \sum^m_{i=1}[y^{(i)}cost_{1}(\theta^{T}x^{(i)}) - (1 - y^{(i)})cost_{0}(\theta^{T}x^{(i)})] + \frac{1}{2} \sum^{n}_{i=1}\theta^{2}_{j}
\end{equation}

## Zadanie 0 

Wczytanie danych ze zbioru oraz wizualizacja.

In [ ]:
import pandas as pd
from pandas.core.interchange.dataframe_protocol import DataFrame

data_input = pd.read_csv('./Ankieta.csv')
data_input.head()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors

x = data_input['plec'].map(lambda x: 1 if x == 'Kobieta' else 0)
y = data_input['waga']
z = data_input['wzrost']

data_input['plec'] = data_input['plec'].map(lambda x: 1 if x == 'Kobieta' else 0)

plt.scatter(y, z, c=x, cmap=colors.ListedColormap(['red', 'green']))
plt.xlabel('waga')
plt.ylabel('wzrost')
plt.title('Ankieta')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (10,10))
ax = fig.gca()
data_input.hist(ax=ax)

In [ ]:
data_input.boxplot()

Na bazie wykresów box-plot można stwierdzić, że dane posiadają różniące się zakresy, co powoduje potrzebę ich skalowania. Warto zauważyć również, że rozkład klas w zbiorze jest równomierny (patrz: histogram)

## Zadanie 1

Proszę dokonać normalizacji zbioru danych za pomocą standaryzacji oraz narysować wykres box-plot dla wszystkich zmiennych. W jaki sposób zmieniły się dane? Co można powiedzieć o ich zakresach. W jakim celu dokonujemy normalizacji?

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

scaler = MinMaxScaler()
scaler.fit(data_input)
std_data_input = scaler.transform(data_input)


fig = plt.figure()
x = std_data_input[:,0]
y = std_data_input[:,1]
z = std_data_input[:,2]
plt.scatter(x[z == 1], y[z == 1],c='blue',label="woman")
plt.scatter(x[z == 0], y[z == 0],c='red',label="men")
plt.xlabel("weight")
plt.ylabel("height")
plt.title("normalized data")
plt.legend()
plt.show()

std_data_input_df = pd.DataFrame(std_data_input, columns=["waga", "wzrost", "plec"])

fig2, ax2 = plt.subplots(figsize=(8,8))
std_data_input_df.hist()

fig3, ax3 = plt.subplots(figsize=(8,8))
std_data_input_df.boxplot(ax=ax3)

## Zadanie 2

W tym zadaniu należy dokonać podziału zbioru danych na uczący oraz testowy. Zbiór uczący będzie służył do treningu klasyfikatora, a testowy do obliczenia ostatecznej skuteczności klasyfikacji. Prosze, by 80% próbek znalazło się w zbiorze uczącym, a 20% w testowym. Proszę zadbać o odpowiednią inicjalizacje generatora pseudolosowego

In [ ]:
from sklearn.model_selection import train_test_split
training_data_st, testing_data_st = train_test_split(std_data_input_df,
                                               test_size=0.2,
                                               train_size=0.8,
                                               random_state=23)

training_data, testing_data = train_test_split(data_input,
                                               test_size=0.2,
                                               train_size=0.8,
                                               random_state=23)

print(training_data_st.shape)
print(testing_data_st.shape)

## Zadanie 3

W tym zadaniu należy dokonać klasyfikacji danych za pomocą klasyfikatora SVC. Proszę obliczyć skuteczność klasyfikatora na danych po, oraz przed standaryzacją i porównać wyniki.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
svc_st = SVC(kernel="rbf", gamma='scale', C=1, random_state=23)
svc = SVC(kernel="rbf", gamma='scale', C=1, random_state=23)
svc_st.fit(training_data_st[["waga","wzrost"]], training_data_st["plec"])
svc.fit(training_data[["waga","wzrost"]], training_data["plec"])

y_pred_st = svc_st.predict(testing_data_st[["waga","wzrost"]])
y_pred = svc.predict(testing_data[["waga","wzrost"]])
print("Accuracy before data standarization:", accuracy_score(testing_data["plec"], y_pred))
print("Accuracy after data standarization:", accuracy_score(testing_data_st["plec"], y_pred_st))


## Zadanie 4

Proszę dobrać odpowiedni parametr C (proszę spróbować z zakresu 0, 5 z krokiem co 0.5). Dla każdego C proszę wyrysować hiperpłaszczyznę utworzoną przez klasyfikator (w formie animimacji, lub inaczej). Proszę przedstawić na wykresie jak zmieniała się skuteczność klasyfikatora w zależności od parametru C. Jakie wnioski można wyciągnąć? Jak wpływa parametr C na wynik?

In [ ]:
import numpy as np
import plotly.graph_objects as go

c_params = np.arange(0.01,5.1,0.5)
print(c_params)

xx, yy = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))

for c in c_params:
    # Train SVM
    svc_st = SVC(kernel="rbf", gamma='scale', C=c, random_state=42)
    svc_st.fit(training_data_st[["waga", "wzrost"]], training_data_st["plec"])

    # Predict on test set
    y_pred_st = svc_st.predict(testing_data_st[["waga", "wzrost"]])
    acc = accuracy_score(testing_data_st["plec"], y_pred_st)
    print(f"Accuracy for C = {c} after standardization: {acc:.4f}")

    # Predict on meshgrid for visualization
    grid_points = pd.DataFrame(np.c_[xx.ravel(), yy.ravel()], columns=["waga", "wzrost"])
    Z = svc_st.predict(grid_points)
    Z = Z.reshape(xx.shape)

    # Separate men and women for coloring
    men = testing_data_st[testing_data_st["plec"] == 0]
    women = testing_data_st[testing_data_st["plec"] == 1]

    # Create scatter plot for men (red)
    scatter_men = go.Scatter3d(
        x=men["waga"],
        y=men["wzrost"],
        z=men["plec"],
        mode='markers',
        marker=dict(size=5, color='red', opacity=0.8),
        name="Men"
    )

    # Create scatter plot for women (blue)
    scatter_women = go.Scatter3d(
        x=women["waga"],
        y=women["wzrost"],
        z=women["plec"],
        mode='markers',
        marker=dict(size=5, color='blue', opacity=0.8),
        name="Women"
    )

    # Create 3D surface plot for decision boundary
    surface = go.Surface(z=Z, x=xx, y=yy, opacity=0.7, colorscale="RdBu", showscale=False)

    # Combine all plots
    fig = go.Figure(data=[surface, scatter_men, scatter_women])
    fig.update_layout(
        title=f"SVM Classification (C = {c})",
        scene=dict(
            xaxis_title="Weight (waga)",
            yaxis_title="Height (wzrost)",
            zaxis_title="Predicted Class (plec)"
        ),
        legend=dict(x=0, y=1)  # Position legend at the top-left
    )
    # Show interactive chart
    fig.show()


'''Dla małych wartości parametru C, algorytm pozwala na większą tolerancję błędów ale lepiej generalizuje, dzięki czemu otrzymujemy większym margines. Dla dużych wartości C algorytm próbuje zklasyfikować wszystkie próbki prawidłowo przez co otrzymujemy wąski margines oraz mniejszą tolerancję błędów.'''

## Zadanie 5

Proszę dokonać pomiaru czasu wykonania algorytmu dla min. 2 różnych kerneli

In [ ]:
import timeit

def train_svc_function(kernel):
    svc_st = SVC(kernel=kernel, gamma='scale', C=c, random_state=42)
    svc_st.fit(training_data_st[["waga", "wzrost"]], training_data_st["plec"])

    y_pred_st = svc_st.predict(testing_data_st[["waga", "wzrost"]])
    accuracy_score(testing_data_st["plec"], y_pred_st)

execution_time = timeit.timeit(lambda: train_svc_function("rbf"), number=500)
print(f"Execution time: {execution_time:.4f} seconds for kernel: rbf")
execution_time = timeit.timeit(lambda: train_svc_function("poly"), number=500)
print(f"Execution time: {execution_time:.4f} seconds for kernel: poly")
execution_time = timeit.timeit(lambda: train_svc_function("sigmoid"), number=500)
print(f"Execution time: {execution_time:.4f} seconds for kernel: sigmoid")

## Zadanie 6

Analiza wektorów nośnych (support vectors). Wyodrębnij wektory nośne z wytrenowanego modelu używając właściwości `.support_vectors_`. Zwizualizuj położenie wektorów nośnych na wykresie, jaki procent danych stanowią wektory nośne?

In [ ]:
svc_st = SVC(kernel="rbf", gamma='scale', C=1, random_state=42)
svc_st.fit(training_data_st[["waga", "wzrost"]], training_data_st["plec"])

s_vectors = svc_st.support_vectors_

total_samples = len(training_data_st)
num_support_vectors = len(s_vectors)
percentage_support_vectors = (num_support_vectors / total_samples) * 100
print(f"Support vectors represent {percentage_support_vectors:.2f}% of the data.")

# Plot the data points and support vectors
plt.scatter(training_data_st['waga'], training_data_st['wzrost'], c=training_data_st['plec'], cmap='coolwarm', label="Data Points")
plt.scatter(s_vectors[:, 0], s_vectors[:, 1], color='black', marker='x', s=100, label="Support Vectors")

plt.xlabel("Weight")
plt.ylabel("Height")
plt.title("Support Vectors in SVM")
plt.legend()
plt.show()

### Dla zbioru *dataR2* proszę dokonać podobnej analizy danych. Opis zbioru: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Coimbra

## Zadanie 7

Proszę zwizualizować dane dla 2 dowolnych zmiennych ze zbioru

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv("dataR2.csv")
# Classification = 1 (Healthy)  / 2 (Patient)
data = data[["Glucose", "Resistin", "Classification"]]
glucose = data["Glucose"]
resistin = data["Resistin"]
classification = data["Classification"]

fig = plt.figure()
plt.scatter(glucose[classification == 1], resistin[classification == 1], c = "green" ,label= "Healthy")
plt.scatter(glucose[classification == 2], resistin[classification == 2], c = "red" ,label= "Patient")
plt.xlabel("Glucose")
plt.ylabel("Resistin")
plt.title("Breast cancer correlation between Glucose and Resistin")
plt.legend()
plt.show()

## Zadanie 8

Proszę dokonać standaryzacji danych

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(data)
std_data = scaler.transform(data)
std_data = pd.DataFrame(std_data, columns=["Glucose", "Resistin", "Classification"])
std_glu = std_data["Glucose"]
std_resi = std_data["Resistin"]
fig = plt.figure()
plt.scatter(std_glu[classification == 1], std_resi[classification == 1], c = "green" ,label= "Healthy")
plt.scatter(std_glu[classification == 2], std_resi[classification == 2], c = "red" ,label= "Patient")
plt.xlabel("Glucose")
plt.ylabel("Resistin")
plt.title("Breast cancer correlation between Glucose and Resistin")
plt.legend()
plt.show()

## Zadanie 9

Trenowanie klasyfikatora. Proszę dokonać treningu klasyfikatora na zbiorze treningowym (X_train, y_train). Proszę użyć różnych wartości parametru C, gamma oraz kernel. Pełna dokumentacja klasyfikatora: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html Wyniki proszę podsumować na odpowiednim wykresie lub tabeli. Test skuteczności klasyfikatora proszę dokonać na zbiorze testowym (X_test, y_test).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

training_data, testing_data = train_test_split(std_data,
                                                test_size=0.2,
                                                random_state=24)

c_params = [0.01, 0.1, 1, 10]
gamma_params = [0.01, 0.1, 1, 10]
kernels = ["rbf","sigmoid","poly"]
results = []

for kern in kernels:
    for c in c_params:
        for g in gamma_params:
            svc = SVC(C=c, kernel=kern, gamma=g)
            svc.fit(training_data[["Glucose","Resistin"]], training_data["Classification"])
            y_predict = svc.predict(testing_data[["Glucose","Resistin"]])
            acc = accuracy_score(testing_data["Classification"], y_predict)
            #print(f"Accuracy: {acc:.3f} for kernel: {kern} C: {c} gamma: {g}")
            result = {
                "kernel": kern,
                "gamma": g,
                "c": c,
                "accuracy": acc
            }
            results.append(result)
df_res = pd.DataFrame(results, columns=["kernel","gamma","c","accuracy"])
df_res

## Zadanie 10

Należy wyznaczyć macierze pomyłek dla klasyfikatora. Proszę dokonać wizualizacji wraz z kolorami na wykresie. Przykłady: 

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix

https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

svc = SVC(C=0.1, kernel="rbf", gamma=10)
svc.fit(training_data[["Glucose","Resistin"]], training_data["Classification"])
y_predict = svc.predict(testing_data[["Glucose","Resistin"]])
y_true = testing_data["Classification"]
cm = confusion_matrix(y_true,y_predict)
print(y_true.shape)

sns.heatmap(cm,
            annot=True,
            fmt="d",
            cmap="magma",
            xticklabels=["Negative", "Positive"],
            yticklabels=["Negative", "Positive"],
            annot_kws={"color": "white"},
            linewidths=1
            )
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()